# 1. Importing libraries 

In [1]:
!pip install pandas
!pip install matplotlib seaborn
import os
from langchain_openai import ChatOpenAI
from crewai import Agent,Task ,Crew, Process
from dotenv import load_dotenv
from crewai_tools import PDFSearchTool
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
load_dotenv()


False

# 2.  DATA Processing

In [2]:
# step1 : Reading HateModerate files 
hate_df = pd.read_csv('all_examples_hate.csv', delimiter='\t')  
nonhate_df = pd.read_csv('all_examples_nonhate.csv', delimiter='\t')  
# setp2 : Randomly select rows
n_rows= 175 # select the sample size you want from each file
sampled1_df = hate_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
sampled2_df = nonhate_df.sample(n=n_rows, random_state=1)  # random_state for reproducibility
# sep 3:  Select the desired columns
result_hate_df = sampled1_df[['example_id', 'sentence']].copy()
result_nonhate_df = sampled2_df[['example_id', 'sentence']].copy()

#step 4 :  Add a new column with the label 'hate'
result_hate_df['label'] = 'hate'
result_nonhate_df['label'] = 'nothate'

# step 5: combin hate and nothate tables in one table
combined_df = pd.concat([result_nonhate_df, result_hate_df], ignore_index=True)

# step 6 :mix the rows 
shuffled_df = combined_df.sample(frac=1, random_state=42).reset_index(drop=True)

## step 7: renaming the final Dataset##
processed_hate_df = shuffled_df 

## step 8 : saving the processed data to a CSV file
processed_hate.to_csv('HateModerate_sample.csv', index=False) 


,example_id,sentence,label
0,11552,"They are more likely to die of coronavirus, we...",nothate
1,11846,I endorse the attacking of muslims,hate
2,7330,I'm sorry to say that I'm prejudiced against p...,hate
3,1802,Hijabis are shit,hate
4,16497,The savages that commit these heinous acts do ...,nothate
...,...,...,...
345,6013,Being a Jew I am a treated like a dirty rat,nothate
346,27958,I don't see myself being friends with people w...,nothate
347,617,sarah do you realise that we live in australia...,hate
348,5146,nigger is illiterate and can't even read or wr...,hate


# 3. Framwork Modelling 

## 3.1 Setting up LLMs

In [3]:
# step1: openai api key#
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY 
#step2: setting up the LLMs that will be used 

llm_gpt4o_mini=ChatOpenAI(model_name="gpt-4o-2024-05-13", temperature=0.0)
llm_gpt4o=ChatOpenAI(model_name="gpt-4o", temperature=0.0)


## 3.2 Designing the tools

In [7]:
## designing RAG PDFsearching tools ## 
group_search = PDFSearchTool('Hate attack types.pdf') # replace with protection level file path
Attacks_search = PDFSearchTool('Hate Slur.pdf') # replace with Hate attack  file path
Slur_search=PDFSearchTool('protection level.pdf') # replace with Hate slur file path

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  1.25it/s]


## 3.3 Designing the four Agents

In [8]:
protection_level_agent=Agent(
    role='English content analyst',
    goal='reading and analyzing {content} and answering questions, passing answers to content moderator',
    memory=False,
    backstory=(
       """ you are an experienced content analyst; you can access protection level documents using the tool,
       Your expertise lies in identifying which groups or individuals are protected and what protection level:
         - fully protected, we called them protected characteristics (PCs).
         - Partially protected, we called them quasi-protected characteristics (Q-PCs).
         - non-protected groups. 
       
       """ 
    ),
    verbose=True,
    max_iter=25,
    allow_delegation=False,
    max_retry_limit=3,
    llm= llm_gpt4o, # <== disable this line if hierarchical is used 
    tools=[group_search]
)

attack_type_agent =Agent(
    role='English Meta content analyst',
    goal='  reading and analyzing {content} and answering questions, passing answers to content moderator',
    memory=False,
    backstory=(
       """ you are an experienced content analyst working for Meta; you can access hate attack types document using the tool,
       Your expertise lies in identifying Hate attacks:
         - tier1 
         - tier2 
         - No Attack
       you also can understand the content and see if: 
         - at least half the group PC or Q-PC were targeted in attacks. 
         - less than half were targeted 
       """ 
    ),
    max_iter=25,
    verbose=True,
    allow_delegation=False,
    max_retry_limit=3,
    llm= llm_gpt4o, # <== disable this line if hierarchical is used 
    tools=[Attacks_search]
)

Hate_slur_agent =Agent(
    role='English content analyst',
    goal='reading and analyzing {content} and answering questions, passing answers to content moderator',
    memory=False,
    backstory=(
       """ you are an experienced content analyst; you can access hate Slur documents using the tool,
       Your expertise lies in identifying: 
       - the hate slurs used
       - hate slur exceptions, if there are any. 
       """ 
    ),
    verbose=True,
    allow_delegation=False,
    max_iter=25,
    max_retry_limit=3,
    llm= llm_gpt4o, # <== disable this line if hierarchical is used 
    tools=[Slur_search]
)
decision_making_agent = Agent(
  role=' Content Moderator',
  goal='Determine if the content contains hate speech based on predefined scenarios and provides a reason',
  backstory="""You are an experienced Content moderator agent, and you have a very good understanding of hate speech scenarios.
                 Your expertise lies in the following: 
                 - content labeling 
                 - identifying the reasons you 
                 """,
  verbose=True,
  allow_delegation=False,
  max_retry_limit=3,
  memory=False,
  max_iter=30,
  llm= llm_gpt4o # <== disable this line if hierarchical is used 
)

## 3.4 Designing the Taskes 

In [9]:
defining_group_task = Task(
  description=""" read the content {content}, and answer the following questions, pass the answers to content_modertor
  Q1- think step by step, Are any groups of people or Human individuals mentioned? 
  note: concepts, institutions, or religions are not considered groups of people or Human individuals   
   for example: 
  *Muslims: are a group, thus Yes group
  *Islam: is a religion, thus No group.  
  *Jewish: is an individual, Yes, Individual. 
  Q2- think step by step, use the group_search tool, Do the group or individuals mentioned have: 
  - protected characteristics (PC)  
  - protected from most severe attacks, quasi-protected Characteristics (Q-PC)
  - None.   
   Examples: 
  *Germans: national origin, thus PC
  *13 years old boys: PC age, thus PC
  *Syrian Refugees: Syrians are of national origin thus, a PC + Refugees describes immigration status. 
                     thus, Syrian Refugees are Q-PC, (PC+Q-PC=Q-PC)
  
  """,
  expected_output=" Yes or No group | PC or Q-PC or None",
  agent=protection_level_agent
)

defining_attack_task = Task(
  description="""read the content {content}, and answer the following questions,  pass the answers to content_modertor
  
  Q1-think step by step; use the Attacks_search tool to answer this question: Have you found any of tier 1 or tier 2 attacks :  
  -  tire 1, define it
  -  tier 2, define it
  -  No Attack. 
    
  Q2-Think step by step, using only your common sense. Has the attack targeted at least half the group? 
  - less than half 
  - half and more 
   examples: 
  "Berliner women are animals": Berliner women are less than half of women in the world; therefore,
  the answer is the attack targeted at less than half.
  "Majority of Russians are not smart": The majority of Russians are more than half of Russians- PC; 
  therefore, the answer would be more than half.
  "many Indians do not wash their hands": many Indians are more than half of them.
  """,
  expected_output=" Tier 1 or tier2 or No Attack | less than half or half and more",
  agent=attack_type_agent 
)


defining_Slur_use_task = Task(
  description="""" read the content {content} use the tool to access Slur defintion,and answer the following questions,  
  pass the answers to content_modertor 
 
  Q1- think step by step, were any Hate slur used?   
  -Hate slur was used; name the slur. 
  -No Hate slur was used
  Note: Hate slurs including like Ethnic slur terms (“nigger”, “kike”, “kraut”) and other group-based slurs (“faggot”, “spaz”)
  must be differentiated from general pejoratives (“asshole”, “idiot”,"mother fucker") and pure expressives (“fuck”) which are not Hate slurs
   Examples: 
    Hate Slurs: Faggot, fag, Nigger, Paki, whitey, nigga .. etc. 
  Q2- think step by step, Was the hate slur used to condemn the use of the slur, report it, self-referential or in an empowering way
  - Yes, Hate slur exception.  
  - NO Hate slur exception.  
  - No Hate slur was used.  
  """,
  expected_output=" Yes or No Hate slur if it was used |  Yes or No, Hate slur exception",
  agent=Hate_slur_agent
)



task_labeling = Task(
  description=""" read the answers provided, label each {content} as hate or not hate, and provide the reason behind your label:

  
   Consider hate speech only in these three Scenarios if you have : 
   - protected characteristics (PC) and [tier1 attack or tier2 attack] and more than half was taregeted .  
   - or if you have Quasi-protected Characteristics (Q-PC) and [tier 1 attack only] and more than half.  
   - or if a Hate slur was used, and NO Hate slur exception.  
   the reason should include if there are any : tier attack type / PC, O-PC or No PC /  the slur if it was used and how it was used. 


  """,
    
  expected_output=""" label | reason 10 words max """,
  
  agent=decision_making_agent
)

## 3.4 Designing the Crew 

In [11]:
## designing sequential_crew
crew_sequential = Crew(
  agents=[protection_level_agent,attack_type_agent,Hate_slur_agent,decision_making_agent],
  tasks=[defining_group_task,defining_attack_task,defining_Slur_use_task,task_labeling],
  process=Process.sequential, 
  full_output=False,

  verbose=2 # You can set it to 1 or 2 to different logging levels 
)

## designing hierarchical crew
crew_hierarchical = Crew(
  agents=[protection_level_agent,attack_type_agent,Hate_slur_agent,decision_making_agent],
  tasks=[defining_group_task,defining_attack_task,defining_Slur_use_task,task_labeling],
  process=Process.hierarchical, 
  manager_llm= llm_gpt4o_mini , #chose the LLM you want to use. 
  full_output=False,

  verbose=2 # You can set it to 1 or 2 to different logging levels 
)

2024-09-24 14:24:28,366 - 29840 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed
2024-09-24 14:24:28,375 - 29840 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


,example_id,sentence,label
0,11552,"They are more likely to die of coronavirus, we...",nothate
1,11846,I endorse the attacking of muslims,hate
2,7330,I'm sorry to say that I'm prejudiced against p...,hate


# 4 Deployment 

In [ ]:

## looping through dataset & saving results in a list##

## step 1: prapearing data for deployment 
Hate_m_df= pd.read_csv('HateModerate_sample.csv')
deplyed_data = processed_hate_df[:10] # here either you can run loop on entire dataframe or slice it like I used only first 10 rows

## step 2 : creating empty list 
label_list = []

##step 3: intinlizing the sequential framework for each sentance 
for i in range(len(deplyed_data)):
    result = crew_sequential.kickoff({"content": str(deplyed_data ["sentence"][i])}) # for hierarchical crew_hierarchical.kickoff
    label_list.append(result)

##step 4:Adding the labels to the hate_df##
deplyed_data["new_label"] = label_list

#step 5: saving new data set on separate file  
deplyed_data.to_csv('data1.csv', index=False) 

#step 6 : reading new data set as CSV file  
df = pd.read_csv('data1.csv')

# step 7:split the 'new_label' into multiple columns new lable and reason
df[['new_label', 'resone']] = df['new_label'].str.split('|', expand=True)

# step 8:  save the final data ready for evaluation
df.to_csv('labeled_data_seq_gpt40.csv', index=False) 




# Evaluation 

In [ ]:
# Step 1: Load the combined CSV file
df = pd.read_csv('the saved filed in deployment pahse path')  # Replace with your actual file name

# Step 2: Convert string labels to binary (1 and 0)
df['label'] = df['label'].map({'hate': 1, 'nothate': 0})          # Replace 'label' with actual column name
df['new_label'] = df['new_label'].map({'hate': 1, 'nothate': 0})  # Replace 'new label' with actual column name

# Step 3: Extract the true labels and predicted labels
y_true = df['label']
y_pred = df['new_label']

# Step 4: Calculate performance metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Step 5: Print the results
print(f'Accuracy: {accuracy:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'F1-Score: {f1:.2f}')

# Step 6: Calculate confusion matrix
cm = confusion_matrix(y_true, y_pred, labels=[1, 0])

# Step 7: Visualize the confusion matrix using Seaborn
plt.figure(figsize=(6, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Predicted Hate', 'Predicted Not Hate'], yticklabels=['Actual Hate', 'Actual Not Hate'])
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.title('Confusion Matrix Hierarchical Process')
plt.show()